# Time-Series Data Pipeline for Operational Monitoring

## Project Overview
Projek simple untuk end-to-end data engineering pipeline dibuat untuk memproses real-world time-series data menggunakan Python dan SQL

## Problem
Data mentah biasanya berisi data yang tidak konsisten dan valuenya mungkin korup (corrupt), membuatnya tidak dapat diandalkan dalam aggregation dan monitoring. Tanpa melalukan validation dan transformation, data tersebut tidak aman untuk digunakan operasional decision-making.

## What Was Built
- Loads/Memuat data time-series mentah dari CSV
- Melakukan data validasi dan menangani numeric values yang korup/corrupted
- Membersihkan dan transformasi data untuk analisi time-series
- Menyimpan hasil data yang telah diproses pada ke relational database (SQLite)
- Memungkinkan aggregasi/aggregation simple berbasis SQL untuk proses pemantauan/monitoring


# ETL

## Data Load

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paulbrabban/daily-minimum-temperatures-in-melbourne")

print("Path to dataset files:", path)

100%|██████████| 14.0k/14.0k [00:00<00:00, 17.1MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/paulbrabban/daily-minimum-temperatures-in-melbourne/versions/2


In [2]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/paulbrabban/daily-minimum-temperatures-in-melbourne/versions/2"

os.listdir(dataset_path)


['daily-minimum-temperatures-in-me.csv']

## Cleaning Validation

In [3]:
import pandas as pd

file_path = dataset_path + "/daily-minimum-temperatures-in-me.csv"

df = pd.read_csv(file_path, on_bad_lines='skip')

df.head()

,Date,"Daily minimum temperatures in Melbourne, Australia, 1981-1990"
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [4]:
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3650 entries, 0 to 3649
Data columns (total 2 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   Date                                                           3650 non-null   object
 1   Daily minimum temperatures in Melbourne, Australia, 1981-1990  3650 non-null   object
dtypes: object(2)
memory usage: 57.2+ KB


,0
Date,0
"Daily minimum temperatures in Melbourne, Australia, 1981-1990",0


In [5]:
!mkdir -p data
!cp "$file_path" data/raw_temperature.csv


In [6]:
df.columns = ["date", "temperature"]

df["date"] = pd.to_datetime(df["date"], errors="coerce")

df = df.dropna()
df = df.sort_values("date")

df.head()


,date,temperature
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [7]:
df[~df["temperature"].astype(str).str.replace('.', '', regex=False).str.isnumeric()].head()


,date,temperature
565,1982-07-20,?0.2
566,1982-07-21,?0.8
1290,1984-07-14,?0.1


In [8]:
df["temperature"] = pd.to_numeric(df["temperature"], errors="coerce")


In [9]:
df = df.dropna(subset=["temperature"])


## Time Series Transform

In [10]:
df["rolling_7d_avg"] = df["temperature"].rolling(window=7).mean()
df["daily_change"] = df["temperature"].diff()

df.head(10)


,date,temperature,rolling_7d_avg,daily_change
0,1981-01-01,20.7,NaN,NaN
1,1981-01-02,17.9,NaN,-2.8
2,1981-01-03,18.8,NaN,0.9
3,1981-01-04,14.6,NaN,-4.2
4,1981-01-05,15.8,NaN,1.2
5,1981-01-06,15.8,NaN,0.0
6,1981-01-07,15.8,17.057143,0.0
7,1981-01-08,17.4,16.585714,1.6
8,1981-01-09,21.8,17.142857,4.4
9,1981-01-10,20.0,17.314286,-1.8


In [11]:
before = len(df)
df = df.dropna(subset=["temperature"])
after = len(df)

print(f"Dropped {before - after} invalid rows")


Dropped 0 invalid rows


In [12]:
!mkdir -p output
df.to_csv("output/cleaned_temperature.csv", index=False)


# SQL **Lite**

## SQL Data Load

In [13]:
import sqlite3

conn = sqlite3.connect("temperature.db")

df.head()


,date,temperature,rolling_7d_avg,daily_change
0,1981-01-01,20.7,NaN,NaN
1,1981-01-02,17.9,NaN,-2.8
2,1981-01-03,18.8,NaN,0.9
3,1981-01-04,14.6,NaN,-4.2
4,1981-01-05,15.8,NaN,1.2


In [14]:
df.to_sql(
    name="daily_temperature",
    con=conn,
    if_exists="replace",
    index=False
)

conn.commit()


In [15]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,daily_temperature


In [16]:
pd.read_sql("SELECT * FROM daily_temperature LIMIT 5;", conn)


,date,temperature,rolling_7d_avg,daily_change
0,1981-01-01 00:00:00,20.7,None,NaN
1,1981-01-02 00:00:00,17.9,None,-2.8
2,1981-01-03 00:00:00,18.8,None,0.9
3,1981-01-04 00:00:00,14.6,None,-4.2
4,1981-01-05 00:00:00,15.8,None,1.2


## SQL aggregation

In [17]:
query_monthly = """
SELECT
  strftime('%Y-%m', date) AS month,
  COUNT(*) AS total_days,
  ROUND(AVG(temperature), 2) AS avg_temp,
  ROUND(MIN(temperature), 2) AS min_temp,
  ROUND(MAX(temperature), 2) AS max_temp
FROM daily_temperature
GROUP BY month
ORDER BY month;
"""

pd.read_sql(query_monthly, conn)


,month,total_days,avg_temp,min_temp,max_temp
0,1981-01,31,17.71,12.1,25.0
1,1981-02,28,17.68,13.7,25.0
2,1981-03,31,13.50,8.0,18.6
3,1981-04,30,12.36,7.4,18.1
4,1981-05,31,9.49,2.1,14.0
...,...,...,...,...,...
115,1990-08,31,7.83,2.1,11.6
116,1990-09,30,9.17,3.0,16.3
117,1990-10,31,11.35,6.0,18.4
118,1990-11,30,12.66,8.3,19.2


In [18]:
query_extreme = """
SELECT
  date,
  temperature,
  daily_change
FROM daily_temperature
WHERE daily_change IS NOT NULL
ORDER BY ABS(daily_change) DESC
LIMIT 5;
"""

pd.read_sql(query_extreme, conn)


,date,temperature,daily_change
0,1984-11-27 00:00:00,11.6,-12.7
1,1985-11-15 00:00:00,10.0,-11.0
2,1985-10-06 00:00:00,6.1,-10.1
3,1987-11-20 00:00:00,14.1,-10.0
4,1981-02-10 00:00:00,15.0,-10.0


In [19]:
query_yearly = """
SELECT
  strftime('%Y', date) AS year,
  ROUND(AVG(temperature), 2) AS avg_temp
FROM daily_temperature
GROUP BY year
ORDER BY year;
"""

pd.read_sql(query_yearly, conn)


,year,avg_temp
0,1981,11.52
1,1982,10.84
2,1983,11.19
3,1984,10.62
4,1985,11.14
5,1986,10.80
6,1987,10.85
7,1988,11.97
8,1989,11.26
9,1990,11.67
